# GROUP PROJECT PROPOSAL

## Introduction:
**Provide some relevant background information on the topic so that someone unfamiliar with it will be prepared to understand the rest of your proposal**

  Classification is the process in which data is grouped into categories based on their properties. Then when presented with new data, we can use the old data to accurately predict the class of the new data. In this proposal we will state the dataset we are going to perform classification on and demonstrate preliminary data analysis while providing context as to how the process will be done and what we are expecting to find. etc.

**Clearly state the question you will try to answer with your project**

  Is there a correlation between the rank, height, age, and the hand of which a player plays? (i.e. we classify if a player is left or right handed based on the three predictors)
	
 **Identify and describe the dataset that will be used to answer the question**

The dataset we will be using is Player Stats for Top 500 Players. 
